In [ ]:
import os
import json
import requests
from datetime import datetime, timedelta
from amadeus import Client, ResponseError
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
from date_time_tool import datetime_tool
from weather_fetcher_tool import weather_tool
from flight_tool import flight_inspiration_tool
# Load environment variables
load_dotenv()


llm = ChatOpenAI(temperature=0.7, openai_api_key=os.getenv("OPENAI_API_KEY"))



/home/zain/my_travel_recommender/flight_tool.py:17: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7, openai_api_key=os.getenv("OPENAI_API_KEY"))


In [2]:
# Example Usage
user_query = "Find me travel destinations from MAD in the next 2 months under 200 dollars"
result = flight_inspiration_tool.run(user_query)

# Output the results
if isinstance(result, dict) and "error" in result:
    print(result["error"])
else:
    for destination in result:
        print(f"Destination: {destination['destination']}, "
              f"Price: ${destination['price']}, "
              f"Departure Date: {destination['departure_date']}")

/home/zain/my_travel_recommender/flight_tool.py:78: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(formatted_prompt)


{'origin': 'MAD', 'departure_date': '2025-03-14', 'max_price': 200}
Destination: LIS, Price: $57.42, Departure Date: 2025-03-14
Destination: PMI, Price: $60.49, Departure Date: 2025-03-14
Destination: OPO, Price: $68.30, Departure Date: 2025-03-14
Destination: LPA, Price: $69.89, Departure Date: 2025-03-14
Destination: MXP, Price: $77.35, Departure Date: 2025-03-14
Destination: FCO, Price: $99.79, Departure Date: 2025-03-14
Destination: ORY, Price: $123.19, Departure Date: 2025-03-14
Destination: RAK, Price: $171.87, Departure Date: 2025-03-14
Destination: LHR, Price: $177.10, Departure Date: 2025-03-14
Destination: AMS, Price: $177.43, Departure Date: 2025-03-14


In [5]:
# Register tools
tools = [flight_inspiration_tool, weather_tool,datetime_tool]

# Create the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

/tmp/ipykernel_90401/879684908.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


In [6]:
query = """
Suggest a travel destination from Madrid with warm weather on 15-02-2025
"""

In [7]:

response = agent.run(query)
print(response)

/tmp/ipykernel_90401/1245204814.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)




> Entering new AgentExecutor chain...
We need to find a destination with warm weather on the specified date. Let's start by fetching the historical weather data for Madrid on 15-02-2025.

Action: Weather Fetcher
Action Input: {"iata_code": "MAD", "target_date": "2025-02-15"}Type of inputs at start: <class 'str'>
Value of inputs at start: {"iata_code": "MAD", "target_date": "2025-02-15"}

Observation: The average temperature at MAD over the last 2 years from 2025-02-15 is 1.48°C.
Thought:Madrid seems to be cold on the specified date. We should now search for flight inspiration to find destinations with warm weather.

Action: Flight Inspiration Search
Action Input: Madrid, 15-02-2025{'origin': 'MAD', 'departure_date': '2025-02-15', 'max_price': None}

Observation: [{'destination': 'PMI', 'price': '50.49', 'departure_date': '2025-02-15'}, {'destination': 'LIS', 'price': '57.42', 'departure_date': '2025-02-15'}, {'destination': 'OPO', 'price': '58.30', 'departure_date': '2025-02-15'}, {'